In [1]:
import os
import sys
import pathlib
import subprocess
import glob

root = pathlib.Path("..").resolve()

fortran_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "MC_singleSeg_singleTS")
sys.path.append(fortran_source_dir)

reservoir_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "Reservoir_singleTS")
sys.path.append(reservoir_source_dir)

routing_v02_dir = os.path.join(root, "src", "python_routing_v02","fast_reach")
sys.path.append(routing_v02_dir)

framework_v02_dir = os.path.join(root, "src", "python_framework_v02")
sys.path.append(framework_v02_dir)

framework_v01_dir = os.path.join(root, "src", "python_framework_v01")
sys.path.append(framework_v01_dir)

# scientific packages
import xarray as xr
import pandas as pd
import numpy as np
import netCDF4
from functools import partial
from itertools import chain, islice

# t-route functions
import nhd_network_utilities_v02 as nnu
import nhd_io
import nhd_network
import network_dl

## Compile cython routing module
- This is specific to A. Wlostowski's system.
- TO DO: Revise for Google Colab - see [this notebook](https://github.com/awlostowski-noaa/t-route/blob/network-compute-notebook/notebooks/compute-network-v2.ipynb)

In [2]:
COMPILE_cython = True
if COMPILE_cython:

    # compile vaPrecision.f90
    subprocess.run(
        [
            "gfortran", 
            "varPrecision.f90", 
            "-c", 
            "-O0", 
            "-fPIC"],
        cwd=fortran_source_dir,
    )

    # compile MCsingleSegStime_f2py_NOLOOP.f90
    subprocess.run(
        [
            "gfortran",
            "-c",
            "-O0",
            "-fPIC",
            "-o",
            "mc_single_seg.o",
            "MCsingleSegStime_f2py_NOLOOP.f90"
        ],
        cwd=fortran_source_dir,
    )

    # compile pyMCsingleSegStime_NoLoop.f90
    subprocess.run(
        [
            "gfortran",
            "pyMCsingleSegStime_NoLoop.f90",
            "-c",
            "-o",
            "pymc_single_seg.o",
            "-O3",
            "-fPIC"
        ],
        cwd=fortran_source_dir,
    )

    # copy *.o files to python_routing_v02/fast_reach 
    for obj_file in glob.glob(os.path.join(fortran_source_dir, "*.o")):
        print(obj_file)
        subprocess.run(["cp", obj_file, routing_v02_dir])    

    # compile vaPrecision.f90
    subprocess.run(
        [
            "gfortran", 
            "varPrecision.f90", 
            "-c", 
            "-O0", 
            "-fPIC"],
        cwd=reservoir_source_dir,
    )

    # compile module_levelpool.f90
    subprocess.run(
        [
            "gfortran",
            "-c",
            "-O2",
            "-fPIC",
            "-o",
            "module_levelpool.o",
            "module_levelpool.f90"
        ],
        cwd=reservoir_source_dir,
    )

    # compile pymodule_levelpool.f90
    subprocess.run(
        [
            "gfortran",
            "-c",
            "-O2",
            "-fPIC",
            "-o",
            "pymodule_levelpool.o",
            "pymodule_levelpool.f90"
        ],
        cwd=reservoir_source_dir,
    ) 

    # copy *.o files to python_routing_v02/fast_reach 
    for obj_file in glob.glob(os.path.join(reservoir_source_dir, "*.o")):
        print(obj_file)
        subprocess.run(["cp", obj_file, routing_v02_dir])

    numpy_I = "/home/awlostowski/tr/lib/python3.8/site-packages/numpy/core/include"
    py_I = "/usr/include/python3.8"
    py_lib = "/home/awlostowski/tr/lib"

    subprocess.run(
        [
            "cython",
            "-3",
            "-v",
            "-p",
            "--gdb",
            "--line-directives",
            "-Wextra",
            "--cleanup",
            "3",
            "fortran_wrappers.pxd"
            "*.pyx"
        ],
        cwd=routing_v02_dir,
    )

    file = ["fortran_wrappers","mc_reach","reservoir","reach","utils"]
    for f in file:
        subprocess.run(
          [
              "gcc",
              "-pthread",
              "-Wno-unused-result",
              "-Wsign-compare",
              "-DNDEBUG",
              "-fwrapv",
              "-O2",
              "-Wall",
              "-Wstrict-prototypes",
              "-fPIC",
              "-fno-strict-aliasing",
              "-I",
              numpy_I,
              "-I",
              py_I,
              "-mtune=generic",
              "-c",
              "-o",
              f + ".o",
              f + ".c"
          ],
          cwd=routing_v02_dir,
        )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-lgfortran",
            "-o",
            "reach.cpython-38-x86_64-linux-gnu.so",
            "mc_single_seg.o",
            "pymc_single_seg.o",
            "fortran_wrappers.o",
            "reach.o"
        ],
        cwd=routing_v02_dir,
    )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-lgfortran",
            "-o",
            "reservoir.cpython-38-x86_64-linux-gnu.so",
            "module_levelpool.o",
            "pymodule_levelpool.o",
            "fortran_wrappers.o",
            "reservoir.o"
        ],
        cwd=routing_v02_dir,
    )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-o",
            "mc_reach.cpython-38-x86_64-linux-gnu.so",
            "mc_reach.o"
        ],
        cwd=routing_v02_dir,
    )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-o",
            "utils.cpython-38-x86_64-linux-gnu.so",
            "utils.o"
        ],
        cwd=routing_v02_dir,
    ) 
    
import mc_reach

/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/mc_single_seg.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/pymc_single_seg.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/varPrecision.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/var_precision.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/Reservoir_singleTS/module_levelpool.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/Reservoir_singleTS/pymodule_levelpool.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/Reservoir_singleTS/varPrecision.o


## Identify supernetwork connections

In [3]:
print("Identifying supernetwork connections set")

test_folder = os.path.join(root, r"test")
geo_input_folder = os.path.join(test_folder, r"input", r"geo")
supernetwork = "Pocono_TEST2"

network_data = nnu.set_supernetwork_data(
    supernetwork=supernetwork, geo_input_folder=geo_input_folder
)

# if the NHDPlus RouteLink file does not exist, download it.
if not os.path.exists(network_data["geo_file_path"]):
    filename = os.path.basename(network_data["geo_file_path"])
    network_dl.download(network_data["geo_file_path"], network_data["data_link"])

# select only the necessary columns of geospatial data, set the DataFrame index
cols = [v for c, v in network_data["columns"].items()]
data = nhd_io.read(network_data["geo_file_path"])
data = data[cols]
data = data.set_index(network_data["columns"]["key"])

# mask NHDNetwork to isolate test network of choice
if "mask_file_path" in network_data:
    data_mask = nhd_io.read_mask(
        network_data["mask_file_path"], layer_string=network_data["mask_layer_string"],
    )
    data = data.filter(data_mask.iloc[:, network_data["mask_key"]], axis=0)

# sort index
data = data.sort_index()

# replace downstreams
data = nhd_io.replace_downstreams(data, network_data["columns"]["downstream"], 0)

# extract downstream connections for each node
connections = nhd_network.extract_connections(data, network_data["columns"]["downstream"])

print("supernetwork connections set created.")

Identifying supernetwork connections set
supernetwork connections set created.


## Organize supernetwork into subnetworks and reaches

In [4]:
print("Organizing segments into reaches.")

# reverse the network - track upstream connections
rconn = nhd_network.reverse_network(connections)

# isolate independent subnetworks
subnets = nhd_network.reachable_network(rconn)

# identify the segments in each subnetwork
subreachable = nhd_network.reachable(rconn)

# break each subnetwork into reaches
subreaches = {}
for tw, net in subnets.items():
    path_func = partial(nhd_network.split_at_junction, net)
    subreaches[tw] = nhd_network.dfs_decomposition(net, path_func)

print("Reach creation complete.")

Organizing segments into reaches.
Reach creation complete.


## Load qlateral data from WRF-Hydro CHRTOUT files
- `nhd_io.get_ql_from_wrf_hydro`

In [116]:
path = os.path.join(root, "test","input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","example_CHRTOUT")
qlat_files = glob.glob(path + "/*", recursive=True)

ql = nhd_io.get_ql_from_wrf_hydro(qlat_files,index_col="feature_id")

# extend ql time series long enough to drain the system
dt_wrf = (ql.columns[1] - ql.columns[0])
time_append = pd.period_range(ql.columns[-1], periods=96, freq=dt_wrf)

for i in time_append.astype(str):
    ql[i] = np.zeros(ql.shape[0])

## Load initial conditions from WRF-hydro HYDRO_RST file
- nhd_io.get_stream_restart_from_wrf_hydro

In [6]:
wrf_hydro_channel_restart_file = os.path.join(root, "test", "input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","example_RESTART",
                    "HYDRO_RST.2017-12-31_06-00_DOMAIN1")
wrf_hydro_channel_ID_crosswalk_file = os.path.join(root, "test", "input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","primary_domain",
                    "DOMAIN","Route_Link.nc")
wrf_hydro_channel_ID_crosswalk_file_field_name = "link"

q0 = nhd_io.get_stream_restart_from_wrf_hydro(
    wrf_hydro_channel_restart_file,
    wrf_hydro_channel_ID_crosswalk_file,
    wrf_hydro_channel_ID_crosswalk_file_field_name
)

## Set-up routing simulation time domain
- The routing simulation should have a total duration equal to the WRF-Hydro run, but a much finer timestep. 
- use the timestamps in `ql.columns` to determine the duration and timestep of the WRF- Hydro run
- User specifies the timestep (seconds) of the routing simulation
- The number of timesteps in the routing simulation is calculated
- _NOTE_ this assumes that the WRF hydro simulation results have a uniform timestep!

In [117]:
dt = 300  # routing simulation timestep (seconds) 
dt_routing = pd.Timedelta(str(dt) + 'seconds')

wrf_time = ql.columns.astype("datetime64[ns]")
dt_wrf = (wrf_time[1] - wrf_time[0])
sim_duration = (wrf_time[-1] + dt_wrf) -wrf_time[0]

nts = round(sim_duration / dt_routing)


## Execute the routing module
- Break here - Wednesday, September 16:50
- TO DO: incorporate changes to mc_reach previously made and now saved in python_routing_v02/mc_reach-anw.pyx
- compare results against streamflow data in CHRTOUT files.

In [118]:
# initialize a results matrix
results = []

# Specify the tailwater of the the test basin
tw = 4186169
reach = subreaches[tw]

# get a list of segments in the subnetwork
r = list(filter(None, chain.from_iterable(reach)))

# add a dt column to the data DataFrame
data["dt"] = dt

# rename columns to specific variable names expected by mc_reach.compute_network
column_rename = {
    network_data["columns"]["dx"]: "dx",
    network_data["columns"]["tw"]: "tw",
    network_data["columns"]["twcc"]: "twcc",
    network_data["columns"]["bw"]: "bw",
    network_data["columns"]["ncc"]: "ncc",
    network_data["columns"]["s0"]: "s0",
    network_data["columns"]["cs"]: "cs",
    network_data["columns"]["n"]: "n",
}

data = data.rename(columns=column_rename)

# change variables to type float32, as expected by mc_reach.compute_network
data = data.astype("float32")

# prep parameter, lateral inflow, and initial condition data to be fed to routing model
data_sub = data.loc[
    r, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
].sort_index()

# prep lateral inflow data
qlat_sub = ql.loc[r].sort_index()

# prep initial conditions data
q0_sub = q0.loc[r].sort_index()
q0_sub_zeros = pd.DataFrame(np.zeros(q0_sub.shape))
q0_sub_zeros.index = q0_sub.index
q0_sub_zeros.columns = q0_sub.columns

# compute the network routing, calculate (flow, depth, and velocity)

# TO DO - pass initial conditions, what to do with q up v. down?
# edit mc_reach to use correct q_lateral data, need to take same value for several timesteps

results.append(
    mc_reach.compute_network(
        nts,
        reach,
        subnets[tw],
        data_sub.index.values,
        data_sub.columns.values,
        data_sub.values.astype("float32"),
        qlat_sub.values.astype("float32"),
        q0_sub_zeros.values.astype("float32"),
        False
    )
)

# create a multi-index DataFrame with flow, depth, and velocity simulations
fdv_columns = pd.MultiIndex.from_product([range(nts), ["q", "v", "d"]])
flowveldepth = pd.concat(
    [pd.DataFrame(d, index=i, columns=fdv_columns) for i, d in results], copy=False
)
flowveldepth = flowveldepth.sort_index()


In [132]:
flows = flowveldepth.loc[:, (slice(None), "q")]
flows = flows.T.reset_index(level = [0,1])
flows.rename(columns = {"level_0": "Timestep", "level_1": "Parameter"}, inplace = True)
flows['Time (d)'] = ((flows.Timestep + 1) * dt)/(24*60*60)
flows = flows.set_index('Time (d)')

flows.to_csv("mc_flow_pocono.csv",index = True)

In [175]:
# create lateral inflow series to print
ql_routing = pd.DataFrame(np.zeros(flows.shape))
ql_routing.index = flows.index
ql_routing.columns = flows.columns

ql_routing["Parameter"] = "qlat"
ql_routing["Timestep"] = flows.Timestep

for i in range(0,nts):
    
    idx_wrf = int(i/(nts/ql.shape[1]))
    
    ql_routing.iloc[i,2:] = ql.iloc[:,idx_wrf]
    
ql_routing.to_csv("qlat_pocono.csv",index = True)


In [215]:
flow = flows.iloc[:,2:]
qlat = ql_routing.iloc[:,2:]

# mass balance
# initial system mass
mass_init = flow.loc[flow.index.min(),:].sum() * dt

# qlateral mass
mass_qlat = qlat.sum().sum() * dt

# mass out
mass_out = flow.loc[:,4186169].sum() * dt

# final system mass
mass_final = flow.loc[flow.index.max(),:].sum() * dt

# mass balance
balance = (mass_qlat - mass_out) + (mass_init - mass_final)
print(balance/mass_qlat)

-0.048560539007942785
